In [2]:
import os
from openai import OpenAI
import pandas as pd
import json

In [3]:
key = os.environ.get("OPENAI_API_KEY")
# if not running locally paste key manually
# DO NOT PUBLISH KEY TO GITHUB
# hard coding creds is poor practice.
# key = ""
client=OpenAI(api_key=key)

In [20]:
import json

def sanitize_to_utf8(input_str):
    """
    Sanitizes a string field to UTF-8.
    """
    return input_str.encode('utf-8', 'replace').decode('utf-8')

def process_jsonl_file(input_jsonl_path, output_jsonl_path):
    """
    Processes a JSONL file line by line, sanitizes each line to UTF-8, 
    and saves it to a new JSONL file.
    """
    with open(input_jsonl_path, 'r', encoding='utf-8') as input_file, \
         open(output_jsonl_path, 'w', encoding='utf-8') as output_file:
        
        for line in input_file:
            # Parse JSON line
            json_obj = json.loads(line)

            # Sanitize data
            sanitized_json_obj = {key: sanitize_to_utf8(value) if isinstance(value, str) else value
                                  for key, value in json_obj.items()}

            # Write sanitized JSON object to new JSONL file
            output_file.write(json.dumps(sanitized_json_obj) + '\n')



Generating intermediate 1500 example sets.


In [21]:
# Paths to your files
input_jsonl_path = 'train_data_full.json'
output_jsonl_path = 'output.jsonl'

# Process the JSONL file
process_jsonl_file(input_jsonl_path, output_jsonl_path)

In [22]:
sys_prompt = "You are an expert capable of discerning truthful from deceptive opinions based on speech patterns."

In [23]:
def gen_finetune(input, output, test=False):
    with open(input, 'r', encoding='utf-8') as data_in, \
        open(output, 'w') as gpt_out:
        for i, line in enumerate(data_in):
            user_prompt = json.loads(line)['text']
            sys_reply = "True" if json.loads(line)["label"] == 1 else "False"
            if not test:
                payload = {"messages": [{"role": "system", "content": sys_prompt}, {"role": "user", "content": user_prompt}, {"role": "assistant", "content": sys_reply}]}
            else:# exclude response from test set
                payload = {"messages": [{"role": "system", "content": sys_prompt}, {"role": "user", "content": user_prompt}]}
            gpt_out.write(json.dumps(payload) + '\n')


In [21]:
# 3 fold CV
gen_finetune('train_data_CV1.json', 'trainCV1_gpt.jsonl')
gen_finetune('val_data_CV1.json', 'valCV1_gpt.jsonl')
gen_finetune('train_data_CV2.json', 'trainCV2_gpt.jsonl')
gen_finetune('val_data_CV2.json', 'valCV2_gpt.jsonl')
gen_finetune('train_data_CV3.json', 'trainCV3_gpt.jsonl')
gen_finetune('val_data_CV3.json', 'valCV3_gpt.jsonl')

In [33]:
gen_finetune('train_data_300.json', 'train300_gpt.jsonl')
gen_finetune('val_data_300.json', 'val300_gpt.jsonl')

In [65]:
gen_finetune('train_data_1500.json', 'train1500_gpt.jsonl')
gen_finetune('val_data_1500.json', 'val1500_gpt.jsonl')

In [20]:
gen_finetune('train_data_full.json', 'train_full_gpt.jsonl')
gen_finetune('val_data_full.json', 'val_full_gpt.jsonl')
#gen_finetune('test_data_full.json', 'test_full_gpt.jsonl')

Example of querying model for classification.  
Example on model trained on 300 example set.

In [8]:
test_lie = "One morning three months ago I was in a hurry and tripped on the steps while running to take a shower before work. I ended up fracturing 4 metatarsal bones that required two surgeries to fix. I was really having a hard time not being able to walk whenever I wanted to. I really had such a bad attitude at the beginning because I was so used to being independent. Now that I have recovered I have a new appreciation for my ability to walk. I feel like the whole time I couldn't walk I was thinking about how much I took that ability for granted. But now I choose to walk more than I ever had before. When I walk the dogs I go further out of my way just to enjoy the ability to do it. I am completely recovered and I am going to take this as a lesson learned. Nothing is more important that my personal health. I need to make sure that even if I am running late, I need to take my time and be careful. Instead f making it to work on time, I ended up missing weeks of work. Now all I do at work is try and catch up with everything I missed. It was really nice that people at work came and visited me at the hospital. I really appreciated all the flowers and candy and food that was delivered. I think that this showed me how loved I truly am."
test_truth = "Each and every abortion is essentially a tragedy. The potential mother will suffer unforeseen consequences. Society as a whole will be deprived of the potential it could have received from the new life."
response = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0613:personal::8S542QSs",
  messages=[
    {"role": "system", "content": sys_prompt},
    {"role": "user", "content": test_lie},
    {"role": "user", "content": test_truth},
  ]
)
response.choices[0].message.content

'False'

In [24]:
mdls = {
    '300':"ft:gpt-3.5-turbo-0613:personal::8S542QSs",
    '3.5':"gpt-3.5-turbo",
    '1500':"ft:gpt-3.5-turbo-0613:personal::8TAkdwiX",
    'CV1':"ft:gpt-3.5-turbo-0613:personal::8ioZ39pz",
    'CV2':"ft:gpt-3.5-turbo-0613:personal::8ipwpWND",
    'CV3':"ft:gpt-3.5-turbo-0613:personal::8ipyhT2n"
    }

In [25]:
def predict(text, model, sys_prompt):
    response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": text}
    ]
    )
    return response.choices[0].message.content

In [74]:
test_truth

'Each and every abortion is essentially a tragedy. The potential mother will suffer unforeseen consequences. Society as a whole will be deprived of the potential it could have received from the new life.'

In [75]:
test_lie

"One morning three months ago I was in a hurry and tripped on the steps while running to take a shower before work. I ended up fracturing 4 metatarsal bones that required two surgeries to fix. I was really having a hard time not being able to walk whenever I wanted to. I really had such a bad attitude at the beginning because I was so used to being independent. Now that I have recovered I have a new appreciation for my ability to walk. I feel like the whole time I couldn't walk I was thinking about how much I took that ability for granted. But now I choose to walk more than I ever had before. When I walk the dogs I go further out of my way just to enjoy the ability to do it. I am completely recovered and I am going to take this as a lesson learned. Nothing is more important that my personal health. I need to make sure that even if I am running late, I need to take my time and be careful. Instead f making it to work on time, I ended up missing weeks of work. Now all I do at work is try 

In [26]:
def eval(test_file, mdl, return_df = False, sys_prompt=sys_prompt, by_class=False):
    msgs = []
    preds = []
    trues = []
    replies = []
    set = []
    with open(test_file, 'r') as f:
        for ex in f:
            ex = json.loads(ex)
            msg = ex['text']
            #pred = predict(msg, mdl) == 'True' #'True' if predict(msg) else 'False'
            reply = predict(msg, mdl, sys_prompt) #for debug
            pred = reply == 'True'
            true = ex['label'] == 1
            msgs.append(msg)
            preds.append(pred)
            trues.append(true)
            replies.append(reply) # debug
            if by_class:
                set.append(ex['set'])
    if by_class:
         test_df = pd.DataFrame.from_dict({'msg': msgs, 'reply':replies, 'preds': preds, 'target': trues, 'set':set})   
    else:
        test_df = pd.DataFrame.from_dict({'msg': msgs, 'reply':replies, 'preds': preds, 'target': trues})   
    acc = (test_df['preds'] == test_df['target']).mean()
    if return_df:
        return (test_df, acc)
    else:
        return acc

Baseline gpt3.5 loves to give verbose or indecisive answers. Verbose answers, that actually classify a sentiment as genuine or deceptive can be classified fairly easily. Howver, there is still the issue of baseline gpt3.5 deciding it does not have enough information to classify the statement.

User: "Each and every abortion is essentially a tragedy. The potential mother will suffer unforeseen consequences. Society as a whole will be deprived of the potential it could have received from the new life."
- "There is no objective truth to the statement as it expresses subjective opinions and beliefs about abortion. It cannot be definitively classified as 'True' or 'False'."

To address this issue we need to engineer a system prompt that discourages this behaivor and adequately explains the task. This prompt will be provided to the model at every example query, so instructions should also be consise to minimize any token overhead that will lead to increased cost of training and queries.

- "You are an expert capable of discerning truthful from deceptive opinions based on speech patterns. Definitively classify the following statement as 'True' or 'False', based on the likelihood  the statement represents a genuinely held beleif or a deception."

This issue is avoided in the finetuned models as the traning process rewards our expected behaivor and output format.

In [21]:
sys_prompt_base = sys_prompt + ' Definitively classify the following statement as \'True\' or \'False\', based on the likelihood  the statement represents a genuinely held beleif or a deception.'
#base_df, base_acc = predict('test_data_300.json', mdls['3.5'])
predict(test_truth, mdls['300'], sys_prompt=sys_prompt)

'False'

In [17]:
test300_df, test300_acc = eval('test_data_300.json', mdls['300'], True, sys_prompt=sys_prompt)
print(test300_acc)
test300_df

0.8222222222222222


,msg,reply,preds,target
0,I do not think that euthanasia should be legal...,False,False,False
1,"I pick up my instrument, hold it in my arms, f...",True,True,True
2,Tomorrow me and my ex partner will be taking o...,True,True,True
3,I wanted to write about one of the best days i...,True,True,False
4,We are going to cannock chase with the mountai...,True,True,True
5,My Step-daughter is having a surprise party fo...,True,True,True
6,Cannibals should be legalized. It is healthier...,False,False,True
7,I don't really care. If people want to get mar...,True,True,True
8,I will get the train to LINCOLN where my niece...,True,True,True
9,I don't think people should be able to end the...,False,False,False


In [34]:
test300_acc

0.8222222222222222

In [32]:
test35_df, test35_acc = eval('test_data_300.json', mdls['3.5'], True, sys_prompt=sys_prompt_base)
print(test35_acc)
test35_df

0.7111111111111111


,msg,reply,preds,true
0,I do not think that euthanasia should be legal...,True,True,False
1,"I pick up my instrument, hold it in my arms, f...",True,True,True
2,Tomorrow me and my ex partner will be taking o...,True,True,True
3,I wanted to write about one of the best days i...,True,True,False
4,We are going to cannock chase with the mountai...,True,True,True
5,My Step-daughter is having a surprise party fo...,True,True,True
6,Cannibals should be legalized. It is healthier...,False,False,True
7,I don't really care. If people want to get mar...,True,True,True
8,I will get the train to LINCOLN where my niece...,True,True,True
9,I don't think people should be able to end the...,True,True,False


In [76]:
len(test1500_df)

1450

In [71]:
# Takes a lot of time, cost = $1
#test1500_df, test1500_acc = eval('test_data_1500.json',mdls['1500'],True,sys_prompt=sys_prompt)
print(test1500_acc)
test1500_df

0.8220689655172414


,msg,reply,preds,target
0,In early June I accompanied my friends to a mu...,True,True,True
1,I was at work when I received a flurry of text...,False,False,False
2,A week ago my husband and I went away for his ...,False,False,False
3,This has been some year for me. When I was a t...,False,False,True
4,I find the idea of gay marriage or same sex ma...,False,False,True
...,...,...,...,...
1445,I have the pleasure of going on a lama walk wh...,True,True,False
1446,About eight months ago I had my first child wi...,False,False,False
1447,We will be going to Regent’s Park for a picnic...,True,True,False
1448,"In May, I tested for my red belt in Tae Kwon D...",True,True,True


In [73]:
test1500_df.to_csv('results1500.csv')

Model / Dataset Bias analysis:

In [57]:
print('Model truth classification rate: ')
print(test300_df['preds'].mean())
print('Dataset truthful statement rate: ')
print(test300_df['true'].mean())

Model truth classification rate: 
0.6222222222222222
Dataset truthful statement rate: 
0.6666666666666666


Model performance by dataset

In [18]:
class_df, class_acc = eval('test_data_1500.json', mdl=mdls['300'], sys_prompt=sys_prompt, return_df=True, by_class=True)

In [19]:
class_df['correct'] = class_df['preds'] == class_df['target']
class_df.groupby('set')['correct'].mean()

set
A    0.791866
B    0.673777
C    0.502128
Name: correct, dtype: float64

In [23]:
class_df.to_csv('results_class.csv')

In [30]:
CV1_df, CV1_acc = eval('CV_test_final.json', mdls['CV1'], return_df=True, by_class=True)

In [31]:
CV1_df['correct'] = CV1_df['preds'] == CV1_df['target']
CV1_df.groupby('set')['correct'].mean()

set
A    0.835
B    0.680
C    0.725
Name: correct, dtype: float64

In [32]:
CV1_acc

0.7466666666666667

In [34]:
CV2_df, CV2_acc = eval('CV_test_final.json', mdls['CV2'], return_df=True, by_class=True)
CV2_acc

0.7516666666666667

In [35]:
CV2_df['correct'] = CV2_df['preds'] == CV2_df['target']
CV2_df.groupby('set')['correct'].mean()

set
A    0.835
B    0.695
C    0.725
Name: correct, dtype: float64

In [36]:
CV3_df, CV3_acc = eval('CV_test_final.json', mdls['CV3'], return_df=True, by_class=True)
CV3_acc

0.735

In [37]:
CV3_df['correct'] = CV3_df['preds'] == CV3_df['target']
CV3_df.groupby('set')['correct'].mean()

set
A    0.800
B    0.705
C    0.700
Name: correct, dtype: float64

In [38]:
base_df, base_acc = eval('CV_test_final.json', mdls['3.5'], return_df=True, by_class=True)
base_acc

0.495

In [39]:
base_df['correct'] = base_df['preds'] == base_df['target']
base_df.groupby('set')['correct'].mean()

set
A    0.495
B    0.475
C    0.515
Name: correct, dtype: float64

In [40]:
base300_df, base300_acc = eval('test_data_300_class.json', mdls['3.5'], return_df=True, by_class=True)
base300_acc

0.4444444444444444

In [41]:
base300_df['correct'] = base300_df['preds'] == base300_df['target']
base300_df.groupby('set')['correct'].mean()

set
A    0.500000
B    0.407407
C    0.500000
Name: correct, dtype: float64

In [4]:
def count_jsonl_elements(file_path):
    count = 0
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            count += 1
    return count

files = [
    'train300_gpt.jsonl', 'val300_gpt.jsonl', 'test_data_300.json',
    'job_train1500_gpt.jsonl', 'job_val1500_gpt.jsonl', 'test_data_1500.json'
         ]
for f in files:
    print(f, count_jsonl_elements(f))


train300_gpt.jsonl 210
val300_gpt.jsonl 45
test_data_300.json 45
job_train1500_gpt.jsonl 6765
job_val1500_gpt.jsonl 1450
test_data_1500.json 1450


Compiling Datafile

In [27]:
print("CV1")
CV1_df_ds, CV1_acc_ds = eval('CV_test_final.json', mdls['CV1'], return_df=True, by_class=True)
print(CV1_acc_ds)
print("CV2")
CV2_df_ds, CV2_acc_ds = eval('CV_test_final.json', mdls['CV2'], return_df=True, by_class=True)
print(CV2_acc_ds)
print("CV3")
CV3_df_ds, CV3_acc_ds = eval('CV_test_final.json', mdls['CV3'], return_df=True, by_class=True)
print(CV3_acc_ds)

CV1
0.745
CV2
0.7516666666666667
CV3
0.745


In [55]:
DATAFILE = pd.concat([
    CV1_df_ds.rename(columns={'msg':'prompt','reply':'mdl1.reply', 'preds':'mdl1.pred'}),
    CV2_df_ds.loc[:,['reply','preds']].rename(columns={'reply':'mdl2.reply', 'preds':'mdl2.pred'}),
    CV3_df_ds.loc[:,['reply','preds']].rename(columns={'reply':'mdl3.reply', 'preds':'mdl3.pred'})
    ], axis=1)
DATAFILE['mdl1.correct'] = DATAFILE['mdl1.pred'] == DATAFILE['target']
DATAFILE['mdl2.correct'] = DATAFILE['mdl2.pred'] == DATAFILE['target']
DATAFILE['mdl3.correct'] = DATAFILE['mdl3.pred'] == DATAFILE['target']
classes = {'A':'opinion', 'B':'memories', 'C':'intention'}
DATAFILE['set'].replace(classes, inplace=True)
DATAFILE['fold'] = 'test'
col_order = ['prompt','set','fold','target','mdl1.reply','mdl1.pred','mdl1.correct','mdl2.reply','mdl2.pred','mdl2.correct','mdl3.reply','mdl3.pred','mdl3.correct']
DATAFILE = DATAFILE[col_order]
folds = pd.read_csv('FOLDS.csv')[['prompt','target','set','fold']]
folds['set'].replace(classes, inplace=True)
DATAFILE = pd.concat([DATAFILE, folds])
DATAFILE.to_csv('DATAFILE.csv')
#DATAFILE.to_excel('DATAFILE.xlsx')
DATAFILE

,prompt,set,fold,target,mdl1.reply,mdl1.pred,mdl1.correct,mdl2.reply,mdl2.pred,mdl2.correct,mdl3.reply,mdl3.pred,mdl3.correct
0,I think people should have the right to euthan...,opinion,test,True,True,True,True,True,True,True,True,True,True
1,I find that it is a terrible perversion. I wou...,opinion,test,True,False,False,False,False,False,False,False,False,False
2,"I am pro Cannabis legalization, especially bec...",opinion,test,True,True,True,True,True,True,True,True,True,True
3,God says gay marriage is a sin and it is. Thos...,opinion,test,False,False,False,True,False,False,True,False,False,True
4,I believe that we should let migrants in witho...,opinion,test,False,False,False,True,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,"As an avid Pokemon player, catching a mewtew i...",intention,3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
896,My boyfriend and I are going to see Chris Rams...,intention,3,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
897,I'm meeting up with Holly and Christy at Holly...,intention,3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
898,I am driving some 2 hours to take the car to a...,intention,3,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
CV1_df.to_csv('dat_mdl1.csv')
CV2_df.to_csv('dat_mdl2.csv')
CV3_df.to_csv('dat_mdl3.csv')